In [1]:
import pandas as pd

data = pd.read_csv("games_data.csv", index_col = 0)
del data["mp.1"]
del data["mp_opp.1"]
del data["index_opp"]
data.sort_values("date")
data.reset_index(drop=True)

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,22.8,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True
1,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,0.842,...,50.0,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,20.0,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False
3,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,0.895,...,28.6,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,16.8,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,0.750,...,34.2,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True
17768,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,0.703,...,25.0,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True
17769,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,0.500,...,25.6,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False
17770,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,0.867,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True


In [2]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

data = data.groupby("team", group_keys=False).apply(add_target)

C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\1746609268.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\1746609268.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\1746609268.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining

In [3]:
data[data["team"] == "PHI"]

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
26,240.0,34.0,80.0,0.425,8.0,28.0,0.286,20.0,22.0,0.909,...,30.9,187.0,105.0,GSW,116,1,2022,2021-11-24,False,False
49,240.0,35.0,82.0,0.427,8.0,25.0,0.320,16.0,21.0,0.762,...,27.6,154.0,110.0,WAS,103,0,2016,2016-02-26,False,True
60,240.0,41.0,85.0,0.482,9.0,20.0,0.450,28.0,35.0,0.800,...,38.0,151.0,121.0,SAC,111,1,2021,2021-02-09,True,True
65,240.0,41.0,84.0,0.488,9.0,30.0,0.300,27.0,31.0,0.871,...,41.2,142.0,130.0,DET,104,0,2021,2021-05-08,True,True
145,240.0,42.0,89.0,0.472,11.0,28.0,0.393,19.0,21.0,0.905,...,30.5,166.0,123.0,ORL,106,0,2019,2019-03-05,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17651,240.0,45.0,90.0,0.500,12.0,25.0,0.480,21.0,28.0,0.750,...,41.9,263.0,127.0,BRK,110,0,2019,2019-03-28,True,True
17658,240.0,34.0,81.0,0.420,12.0,27.0,0.444,24.0,30.0,0.800,...,28.6,227.0,110.0,UTA,97,1,2018,2017-11-07,True,False
17669,240.0,34.0,85.0,0.400,13.0,33.0,0.394,18.0,26.0,0.692,...,50.6,252.0,109.0,CHO,119,0,2016,2016-03-02,False,False
17670,240.0,38.0,82.0,0.463,9.0,31.0,0.290,32.0,45.0,0.711,...,39.9,140.0,117.0,ATL,127,1,2020,2020-01-30,False,False


In [4]:
data["target"][pd.isnull(data["target"])] = 2

C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\1494778666.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["target"][pd.isnull(data["target"])] = 2
C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\1494778666.py:1: SettingWithCopy

In [5]:
data["target"] = data["target"].astype(int, errors="ignore")

In [6]:
data["won"].value_counts()

won
True     8886
False    8886
Name: count, dtype: int64

In [7]:
nulls = pd.isnull(data)
nulls = nulls.sum()
nulls = nulls[nulls > 0]
nulls

+/-             17772
mp_max          17772
mp_max.1        17772
+/-_opp         17772
mp_max_opp      17772
mp_max_opp.1    17772
dtype: int64

In [8]:
valid_columns = data.columns[~data.columns.isin(nulls.index)]

In [9]:
valid_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'usg%_max_opp', 'ortg_max_opp', 'drtg_max_opp', 'team_opp', 'total_opp',
       'home_opp', 'season', 'date', 'won', 'target'],
      dtype='object', length=142)

In [10]:
data = data[valid_columns].copy()
data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won,target
0,240.0,39.0,81.0,0.481,6.0,20.0,0.300,14.0,18.0,0.778,...,29.0,178.0,111.0,DAL,95,1,2016,2015-12-09,True,1
1,240.0,36.0,100.0,0.360,7.0,31.0,0.226,16.0,19.0,0.842,...,32.6,152.0,111.0,ATL,98,0,2016,2015-12-09,False,0
2,240.0,37.0,85.0,0.435,8.0,19.0,0.421,17.0,23.0,0.739,...,30.9,148.0,116.0,SAS,107,1,2018,2017-10-18,False,1
3,240.0,41.0,89.0,0.461,8.0,21.0,0.381,17.0,19.0,0.895,...,30.9,138.0,118.0,MIN,99,0,2018,2017-10-18,True,0
4,240.0,27.0,86.0,0.314,6.0,26.0,0.231,15.0,20.0,0.750,...,30.9,157.0,90.0,MEM,92,1,2021,2021-04-30,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,35.0,81.0,0.432,11.0,26.0,0.423,27.0,36.0,0.750,...,33.7,160.0,118.0,OKC,92,0,2019,2018-10-19,True,1
17768,240.0,37.0,74.0,0.500,13.0,25.0,0.520,26.0,37.0,0.703,...,30.0,139.0,129.0,ORL,108,1,2017,2016-12-14,True,2
17769,240.0,42.0,89.0,0.472,14.0,33.0,0.424,10.0,20.0,0.500,...,29.9,175.0,126.0,LAC,113,0,2017,2016-12-14,False,2
17770,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,0.867,...,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True,2


In [11]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction="forward", cv=split)

In [12]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [13]:
selected_columns = data.columns[~data.columns.isin(removed_columns)]

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data[selected_columns] = scaler.fit_transform(data[selected_columns])

In [15]:
sfs.fit(data[selected_columns], data["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [16]:
predictors = list(selected_columns[sfs.get_support()])
predictors

['drb',
 'pf',
 'blk%',
 'usg%',
 'fg_max',
 'orb_max',
 'ts%_max',
 'efg%_max',
 '3par_max',
 'trb%_max',
 'ast%_max',
 'blk%_max',
 'drtg_max',
 '3p_opp',
 '3pa_opp',
 '3p%_opp',
 'fta_opp',
 'ftr_opp',
 'ast%_opp',
 'usg%_opp',
 '3pa_max_opp',
 '3p%_max_opp',
 'fta_max_opp',
 'ast_max_opp',
 'tov_max_opp',
 'pts_max_opp',
 'ts%_max_opp',
 'drb%_max_opp',
 'ast%_max_opp',
 'stl%_max_opp']

In [17]:
def backtest(ds, model, predictors, start=2, step=1):
    all_predcitions = []

    seasons = sorted(ds["season"].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]

        train = ds[ds["season"] < season]
        test = ds[ds["season"] == season]

        model.fit(train[predictors], train["target"])
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)

        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predcitions.append(combined)
    return pd.concat(all_predcitions)

In [18]:
predictions = backtest(data, rr, predictors)

In [19]:
predictions

,actual,prediction
2,1,0
3,0,1
24,0,1
25,1,0
38,0,1
...,...,...
17721,1,0
17748,0,1
17749,2,1
17758,2,1


In [20]:
# calculate how accurate the predictions are
from sklearn.metrics import accuracy_score

accuracy_score(predictions["actual"], predictions["prediction"])

0.505510301868711

In [21]:
data.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\2735305868.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data.groupby("home").apply(lambda x: x[x["won"] == 1].shape[0] / x.shape[0])


home
0.0    0.428314
1.0    0.571686
dtype: float64

In [23]:
data_rolling = data[list(selected_columns) + ["won", "team", "season"]]
data_rolling

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,won,team,season
0,0.0,0.454545,0.308824,0.507177,0.206897,0.242424,0.356295,0.302326,0.269841,0.740957,...,0.044,0.190776,0.088575,0.421801,0.400000,0.298077,1.0,True,ATL,2016
1,0.0,0.386364,0.588235,0.217703,0.241379,0.409091,0.268409,0.348837,0.285714,0.815636,...,0.062,0.475891,0.134788,0.298578,0.400000,0.326923,0.0,False,DAL,2016
2,0.0,0.409091,0.367647,0.397129,0.275862,0.227273,0.500000,0.372093,0.349206,0.695449,...,0.076,0.161426,0.112965,0.279621,0.458824,0.413462,1.0,False,MIN,2018
3,0.0,0.500000,0.426471,0.459330,0.275862,0.257576,0.452494,0.372093,0.285714,0.877480,...,0.050,0.251572,0.112965,0.232227,0.482353,0.336538,0.0,True,SAS,2018
4,0.0,0.181818,0.382353,0.107656,0.206897,0.333333,0.274347,0.325581,0.301587,0.708285,...,0.113,0.127883,0.112965,0.322275,0.152941,0.269231,1.0,False,ORL,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.063,0.310273,0.148909,0.336493,0.482353,0.269231,0.0,True,LAC,2019
17768,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.038,0.213836,0.101412,0.236967,0.611765,0.423077,1.0,True,LAC,2017
17769,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.074,0.220126,0.100128,0.407583,0.576471,0.471154,0.0,False,ORL,2017
17770,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.064,0.242138,0.064185,0.289100,0.576471,0.403846,1.0,True,BOS,2020


In [26]:
# find team performance in the last 5 games to predict their outcome for the next game
def find_team_averages(team):
    numeric_cols = team.select_dtypes(include=['number'])
    rolling = numeric_cols.rolling(5).mean()
    return team.loc[:, team.columns.difference(numeric_cols.columns)].join(rolling)

# group so that we only use data from the same season to predict
data_rolling = data_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\2904706620.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_rolling = data_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [28]:
data_rolling

,team,won,mp,fg,fga,fg%,3p,3pa,3p%,ft,...,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,season
0,ATL,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MIN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SAS,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ORL,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,LAC,True,0.05,0.418182,0.391176,0.384211,0.386207,0.384848,0.451781,0.530233,...,0.344266,0.0510,0.0652,0.254088,0.156611,0.316588,0.449412,0.455769,0.4,2019.0
17768,LAC,True,0.00,0.445455,0.244118,0.554545,0.400000,0.324242,0.547981,0.534884,...,0.341743,0.0548,0.0542,0.290985,0.132734,0.370616,0.583529,0.425000,0.4,2017.0
17769,ORL,False,0.10,0.481818,0.435294,0.422967,0.337931,0.351515,0.427078,0.334884,...,0.321789,0.0556,0.0570,0.290776,0.138896,0.436967,0.421176,0.442308,0.4,2017.0
17770,BOS,True,0.00,0.445455,0.426471,0.395215,0.413793,0.475758,0.403800,0.390698,...,0.210780,0.0424,0.0934,0.318449,0.091913,0.388626,0.491765,0.430769,0.6,2020.0


In [29]:
rolling_cols = [f"{col}_5" for col in data_rolling.columns]
data_rolling.columns = rolling_cols

data = pd.concat([data, data_rolling], axis=1)

In [30]:
data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast%_max_opp_5,stl%_max_opp_5,blk%_max_opp_5,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,season_5
0,0.0,0.454545,0.308824,0.507177,0.206897,0.242424,0.356295,0.302326,0.269841,0.740957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.386364,0.588235,0.217703,0.241379,0.409091,0.268409,0.348837,0.285714,0.815636,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.409091,0.367647,0.397129,0.275862,0.227273,0.500000,0.372093,0.349206,0.695449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,0.500000,0.426471,0.459330,0.275862,0.257576,0.452494,0.372093,0.285714,0.877480,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.181818,0.382353,0.107656,0.206897,0.333333,0.274347,0.325581,0.301587,0.708285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.344266,0.0510,0.0652,0.254088,0.156611,0.316588,0.449412,0.455769,0.4,2019.0
17768,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.341743,0.0548,0.0542,0.290985,0.132734,0.370616,0.583529,0.425000,0.4,2017.0
17769,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.321789,0.0556,0.0570,0.290776,0.138896,0.436967,0.421176,0.442308,0.4,2017.0
17770,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.210780,0.0424,0.0934,0.318449,0.091913,0.388626,0.491765,0.430769,0.6,2020.0


In [31]:
data = data.dropna()

In [32]:
data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast%_max_opp_5,stl%_max_opp_5,blk%_max_opp_5,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,season_5
215,0.0,0.750000,0.455882,0.722488,0.241379,0.242424,0.415677,0.279070,0.333333,0.522754,...,0.328440,0.0656,0.0882,0.272746,0.158408,0.373460,0.465882,0.396154,0.4,2021.0
224,0.0,0.431818,0.470588,0.344498,0.172414,0.196970,0.349169,0.488372,0.380952,0.859977,...,0.343349,0.0392,0.0582,0.311530,0.113222,0.297630,0.369412,0.421154,0.6,2021.0
274,0.0,0.613636,0.367647,0.650718,0.655172,0.545455,0.564133,0.465116,0.412698,0.740957,...,0.266514,0.0740,0.0684,0.440042,0.199487,0.324171,0.567059,0.375000,0.6,2016.0
286,0.0,0.545455,0.397059,0.538278,0.275862,0.303030,0.395487,0.325581,0.365079,0.562427,...,0.330275,0.0792,0.0716,0.300210,0.107574,0.397156,0.562353,0.392308,0.8,2016.0
307,0.0,0.340909,0.308824,0.361244,0.448276,0.409091,0.497625,0.488372,0.460317,0.688448,...,0.231422,0.0550,0.0730,0.245493,0.174069,0.329858,0.517647,0.342308,0.2,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.344266,0.0510,0.0652,0.254088,0.156611,0.316588,0.449412,0.455769,0.4,2019.0
17768,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.341743,0.0548,0.0542,0.290985,0.132734,0.370616,0.583529,0.425000,0.4,2017.0
17769,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.321789,0.0556,0.0570,0.290776,0.138896,0.436967,0.421176,0.442308,0.4,2017.0
17770,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.210780,0.0424,0.0934,0.318449,0.091913,0.388626,0.491765,0.430769,0.6,2020.0


In [36]:
# use home court advantage to improve predictions' accuracy
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(data, col_name):
    return data.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

data["home_next"] = add_col(data, "home")
data["team_opp_next"] = add_col(data, "team_opp")
data["date_next"] = add_col(data, "date")

C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\2419348995.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return data.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))
C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\2419348995.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["home_next"] = add_col(data, "home")
C:\Users\tuand\AppData\Local\Temp\ipykernel_17616\2419348995.py:7: DeprecationWarning: DataFrameGroupBy.app

In [39]:
data

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,tov%_max_opp_5,usg%_max_opp_5,ortg_max_opp_5,drtg_max_opp_5,total_opp_5,home_opp_5,season_5,home_next,team_opp_next,date_next
215,0.0,0.750000,0.455882,0.722488,0.241379,0.242424,0.415677,0.279070,0.333333,0.522754,...,0.272746,0.158408,0.373460,0.465882,0.396154,0.4,2021.0,0.0,OKC,2021-01-13
224,0.0,0.431818,0.470588,0.344498,0.172414,0.196970,0.349169,0.488372,0.380952,0.859977,...,0.311530,0.113222,0.297630,0.369412,0.421154,0.6,2021.0,1.0,UTA,2021-02-27
274,0.0,0.613636,0.367647,0.650718,0.655172,0.545455,0.564133,0.465116,0.412698,0.740957,...,0.440042,0.199487,0.324171,0.567059,0.375000,0.6,2016.0,0.0,MIN,2016-03-21
286,0.0,0.545455,0.397059,0.538278,0.275862,0.303030,0.395487,0.325581,0.365079,0.562427,...,0.300210,0.107574,0.397156,0.562353,0.392308,0.8,2016.0,0.0,OKC,2021-10-26
307,0.0,0.340909,0.308824,0.361244,0.448276,0.409091,0.497625,0.488372,0.460317,0.688448,...,0.245493,0.174069,0.329858,0.517647,0.342308,0.2,2018.0,0.0,OKC,2021-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,0.0,0.363636,0.308824,0.389952,0.379310,0.333333,0.502375,0.604651,0.555556,0.708285,...,0.254088,0.156611,0.316588,0.449412,0.455769,0.4,2019.0,0.0,ORL,2016-12-14
17768,0.0,0.409091,0.205882,0.552632,0.448276,0.318182,0.617577,0.581395,0.571429,0.653442,...,0.290985,0.132734,0.370616,0.583529,0.425000,0.4,2017.0,NaN,None,None
17769,0.0,0.522727,0.426471,0.485646,0.482759,0.439394,0.503563,0.209302,0.301587,0.416569,...,0.290776,0.138896,0.436967,0.421176,0.442308,0.4,2017.0,NaN,None,None
17770,0.0,0.500000,0.367647,0.509569,0.310345,0.333333,0.410926,0.581395,0.460317,0.844807,...,0.318449,0.091913,0.388626,0.491765,0.430769,0.6,2020.0,NaN,None,None


In [44]:
# gather stats about opponent team (previous 5 games)
full = data.merge(data[rolling_cols + ["team_opp_next", "date_next", "team"]], left_on=["team", "date_next"], right_on=["team_opp_next", "date_next"])
full

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,blk%_max_opp_5_y,tov%_max_opp_5_y,usg%_max_opp_5_y,ortg_max_opp_5_y,drtg_max_opp_5_y,total_opp_5_y,home_opp_5_y,season_5_y,team_opp_next_y,team_y
0,0.00,0.500000,0.514706,0.389952,0.344828,0.469697,0.339667,0.441860,0.333333,0.893816,...,0.0752,0.290147,0.154300,0.332701,0.538824,0.407692,0.6,2021.0,SAS,LAL
1,0.00,0.636364,0.514706,0.540670,0.586207,0.500000,0.545131,0.372093,0.333333,0.735123,...,0.0922,0.512788,0.197433,0.410427,0.529412,0.536538,0.8,2021.0,LAL,SAS
2,0.00,0.431818,0.514706,0.313397,0.586207,0.651515,0.429929,0.232558,0.253968,0.588098,...,0.0710,0.296226,0.132991,0.472038,0.491765,0.334615,0.6,2022.0,MIN,MIA
3,0.00,0.136364,0.279412,0.112440,0.137931,0.363636,0.169834,0.558140,0.412698,0.913652,...,0.0678,0.430818,0.172786,0.478673,0.475294,0.417308,0.6,2018.0,MIL,CHO
4,0.00,0.340909,0.411765,0.279904,0.103448,0.287879,0.154394,0.279070,0.253968,0.725788,...,0.1134,0.249895,0.174069,0.502370,0.531765,0.434615,0.2,2018.0,IND,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16711,0.25,0.522727,0.544118,0.392344,0.275862,0.378788,0.327791,0.465116,0.476190,0.623104,...,0.1144,0.337526,0.193582,0.345024,0.430588,0.444231,0.6,2017.0,MIL,NYK
16712,0.25,0.409091,0.485294,0.308612,0.241379,0.272727,0.377672,0.581395,0.507937,0.752625,...,0.1060,0.365618,0.199487,0.701422,0.529412,0.465385,0.4,2020.0,PHO,DET
16713,0.00,0.454545,0.250000,0.566986,0.310345,0.272727,0.485748,0.558140,0.412698,0.913652,...,0.0348,0.413208,0.136072,0.400948,0.407059,0.367308,0.2,2016.0,MEM,LAC
16714,0.00,0.568182,0.294118,0.672249,0.482759,0.469697,0.475059,0.372093,0.269841,0.934656,...,0.1138,0.407757,0.149936,0.374408,0.456471,0.378846,0.6,2018.0,LAC,OKC


In [45]:
# test merge
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,SAS,LAL,LAL,SAS,2021-01-07
1,LAL,SAS,SAS,LAL,2021-01-07
2,MIN,MIA,MIA,MIN,2018-12-30
3,MIL,CHO,CHO,MIL,2017-10-23
4,IND,BOS,BOS,IND,2017-12-18
...,...,...,...,...,...
16711,MIL,NYK,NYK,MIL,2018-12-25
16712,PHO,DET,DET,PHO,2022-01-16
16713,MEM,LAC,LAC,MEM,2021-02-26
16714,LAC,OKC,OKC,LAC,2018-10-19


In [47]:
# filter excessive data
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns
removed_columns

['team_x',
 'team_opp',
 'date',
 'team_5_x',
 'team_opp_next_x',
 'date_next',
 'team_5_y',
 'team_opp_next_y',
 'team_y',
 'team_x',
 'team_opp',
 'date',
 'team_5_x',
 'team_opp_next_x',
 'date_next',
 'team_5_y',
 'team_opp_next_y',
 'team_y',
 'season',
 'date',
 'won',
 'target',
 'team',
 'team_opp']

In [49]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]
selected_columns

Index(['mp', 'fg', 'fga', 'fg%', '3p', '3pa', '3p%', 'ft', 'fta', 'ft%',
       ...
       'ast%_max_opp_5_y', 'stl%_max_opp_5_y', 'blk%_max_opp_5_y',
       'tov%_max_opp_5_y', 'usg%_max_opp_5_y', 'ortg_max_opp_5_y',
       'drtg_max_opp_5_y', 'total_opp_5_y', 'home_opp_5_y', 'season_5_y'],
      dtype='object', length=413)

In [50]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [51]:
predictors = list(selected_columns[sfs.get_support()])
predictors

['efg%',
 'orb%',
 'usg%',
 'drb%_max',
 'stl%_max',
 'blk%_max',
 'home',
 '3pa_opp',
 'drb%_opp',
 'usg%_opp',
 'fg_max_opp',
 'ast%_max_opp',
 'home_opp',
 'usg%_5_x',
 'fg%_max_5_x',
 'fg_opp_5_x',
 'pts_opp_5_x',
 'usg%_opp_5_x',
 'ft_max_opp_5_x',
 'pf_max_opp_5_x',
 'total_opp_5_x',
 'home_next',
 'usg%_5_y',
 'orb%_max_5_y',
 'stl%_max_5_y',
 'ortg_max_5_y',
 'ftr_opp_5_y',
 'usg%_opp_5_y',
 'fta_max_opp_5_y',
 'blk%_max_opp_5_y']

In [53]:
predictions = backtest(full, rr, predictors)
predictions

,actual,prediction
3,0,1
4,0,1
5,0,0
6,1,0
8,1,1
...,...,...
16656,0,0
16686,1,0
16687,0,0
16688,1,1


In [55]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.5702022777494475

In [ ]:
# accuracy increases from 0.50 to 0.57